In [89]:
import pandas as pd

movies = pd.read_excel('dev_set_groundtruth_and_trailers.xls',index_col='filename')


In [90]:
import os
import xml.etree.cElementTree as et

for filename in os.listdir('XML'):
    tree = et.parse('XML/'+filename)
    root = tree.getroot()
    metadata = root[0]
    
    for i in metadata.attrib:
        movies.loc[filename[:-4],i] = metadata.get(i)

In [91]:
movies.drop(['Website','imdbID','poster','tomatoConsensus','trailer','writer','DVD','plot'], axis=1, inplace=True)
#TODO: Implement solutions for this:
movies.drop(['tomatoImage','released','year','awards','runtime','title'], axis=1, inplace=True)

In [92]:
movies[['BoxOffice']] = movies[['BoxOffice']].replace('[\$,.]','',regex=True)
movies[['BoxOffice']] = movies[['BoxOffice']].replace(to_replace='M',value='00000',regex=True)
movies[['BoxOffice']] = movies[['BoxOffice']].replace(to_replace='k',value='00',regex=True)
movies[['BoxOffice']] = movies[['BoxOffice']].replace('N/A','NaN').astype(float)


In [93]:
movies['imdbRating'] = movies['imdbRating'].astype(float)
movies['imdbVotes'] = movies['imdbVotes'].replace('[,]','',regex=True).astype(float)
movies = movies.replace('N/A','NaN')


In [94]:
import numpy as np

def oneHotEncodeMultipleValuesInOneEntry(column_name):

    temp = set()
    
    def getPossibleValues(input_string, output_set):
        cur = input_string.split(', ')
        for i in cur:
            output_set.add(i)
    
    movies[[column_name]].applymap(lambda x: getPossibleValues(x,temp))
    
    for i in temp:
        movies[i] = np.zeros(95)
        
    counter = 0
    for i in movies.loc[:,column_name]:
        cur = i.split(', ')
        for j in cur:
            movies.ix[counter,j] += 1
        counter += 1

In [95]:
oneHotEncodeMultipleValuesInOneEntry('country')
oneHotEncodeMultipleValuesInOneEntry('actors')
oneHotEncodeMultipleValuesInOneEntry('genre')
oneHotEncodeMultipleValuesInOneEntry('director')
oneHotEncodeMultipleValuesInOneEntry('type')
oneHotEncodeMultipleValuesInOneEntry('rated')
oneHotEncodeMultipleValuesInOneEntry('language')
oneHotEncodeMultipleValuesInOneEntry('Production')


movies.drop(['actors','country','genre','director','type','rated','language','Production'], axis=1, inplace=True)

C:\ProgramData\Anaconda3\envs\Experimental_Design_for_Data_Science\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
C:\ProgramData\Anaconda3\envs\Experimental_Design_for_Data_Science\lib\site-packages\pandas\core\indexing.py:961: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  return getattr(section, self.name)[new_key]


In [99]:
from sklearn.impute import SimpleImputer

movies = movies.astype(float)
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit_transform(movies)

array([[ 1.        ,  1.        , 80.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        ,  0.        , 64.09333333, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        ,  1.        , 44.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.        ,  0.        , 67.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        ,  0.        , 77.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        ,  1.        , 49.        , ...,  0.        ,
         0.        ,  0.        ]])

In [97]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, Y_train, Y_test = train_test_split(movies.drop(['goodforairplane'],axis=1),movies.loc[:,'goodforairplane'])

knn = KNeighborsClassifier()
knn.fit(X_train,Y_train)
Y_pred = knn.predict(X_test)
print(classification_report(Y_test,Y_pred))


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').